# Main

In [2]:
import dfBasics
import pandas as pd

version_sla = 'v00001'
version     = version_sla + '/v00000'

home_directory  =  '/home/jovyan/work/'
#share_directory =  '/home/jovyan/work/share/'
share_directory =  '/home/jovyan/share/'

In [3]:
sparkSession = dfBasics.getSparkSession()

In [4]:
#senders = list(sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/v00000/v00000/senders/senders.parquet').toPandas()['CSENDERENDPOINTID'])

In [6]:
#senders

In [7]:
senders_files = sparkSession.read.options(delimiter=',') \
                      .csv('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/senders/senders_files.txt').toPandas()
senders_files.columns = ['size','filename']    

In [20]:
filename = senders_files['filename'][0]
failed = []
for filename in senders_files['filename']:
    try:
        sparkSession.read.text('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/encoded/senders/' + filename + '/_SUCCESS')
    except Exception as exception: 
        if not 'temporary' in filename:
            failed.append(filename.split('sla_enc_')[1].split('.')[0])

In [21]:
failed

['069436a0-a16b-11e9-a189-ddafac1b495c',
 '24345d10-b904-11e9-af3f-21adac1b495c',
 '2b11b860-5c0b-11ec-81bf-11c3ac1e124d',
 '40d50320-f563-11e9-9e7f-fa2cac1b495c',
 '7309fb90-f1a6-11e9-8b88-0b60ac1b495c',
 'bbaed6b1-437e-11eb-b000-d76cac1b495c_0ef2c320-3e18-11eb-97db-29f9ac1b495c',
 'c5956260-e0ee-11e8-be62-528eac1b495c',
 'e1b2df80-a796-11e9-b569-14e0ac1b495c_a771d9a0-869b-11e9-b822-0ff9ac1b495c']

# Examples

## some info

In [ ]:
len(senders_files),max(senders_files['size'])

## get message tracking data 

### single file

In [ ]:
def get_empty_message_tracking_dataframe():
    columns = ['CGLOBALMESSAGEID', 'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE', \
       'CSENDERENDPOINTID', 'CSENDERPROTOCOL', 'CINBOUNDSIZE', \
       'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT', 'CMESSAGETAT2', \
       'CSLADELIVERYTIME', 'year', 'month', 'day', 'hour', 'minute']
    return pd.DataFrame(columns=columns)

def get_message_tracking_dataframe_of_senders_filename(filename, version=version):
    try:
        return sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/encoded/senders/' + filename).toPandas()
    except Exception as exception: 
        return get_empty_message_tracking_dataframe()

In [ ]:
pfall = get_message_tracking_dataframe_of_senders_filename(senders_files['filename'][1])    

In [ ]:
pfall.head()

In [ ]:
len(pd.unique(pfall['CGLOBALMESSAGEID'])),len(pfall), pd.unique(pfall['CSTATUS']), pd.unique(pfall['CSERVICE'])

In [ ]:
pfall[ pfall['CSERVICE'] == -1]

### create parquet file with all senders

In [5]:
df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/encoded/senders/*')
df.write.mode("overwrite").parquet("/home/jovyan/work/output/sla_enc_all" + ".parquet")
#pfall = df.toPandas()

### all senders

In [7]:
df = sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/encoded/sla_enc_all' + ".parquet")

In [8]:
pfall = df.toPandas()

Py4JJavaError: An error occurred while calling o72.collectToPython.
: java.lang.OutOfMemoryError: Java heap space


## check message tracking files

In [ ]:
def get_skip_senders(senders_files):
    skip_senders_files = []
    for file in senders_files:
        try:
            pfall = sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/encoded/senders/' + file).toPandas()
            if -1 in pd.unique(pfall['CSENDERENDPOINTID']) :
                skip_senders_files.append(file)
        except Exception as exception:     
            skip_senders_files.append(file)

In [ ]:
skip_senders_files = get_skip_senders(senders_files['filename'])
tested_senders_files = [file for file in senders_files['filename'] if file not in skip_senders_files]
#len(skip_senders_files)

## encoded values

## init encoders (run only once)

In [ ]:
# - run this paragraph only once !
# - if you get an 'allow_pickle' error you need Kernel/restart kernel

import encoder
import numpy as np

np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# restore np.load for future normal usage
#np.load = np_load_old

### encode value

In [ ]:
def transform(value,column=None, npy= share_directory + 'sla/' + version_sla + '/npy'):
    _encoder = encoder.TolerantLabelEncoder(ignore_unknown=True)
    _encoder.classes_ = np.load(npy + '/' + column + '.npy')
    return int( _encoder.transform([value])[0])

### decode value

In [ ]:
def inverse_transform(value,column=None, npy= share_directory + 'sla/' + version_sla + '/npy'):
    _encoder = encoder.TolerantLabelEncoder(ignore_unknown=True)
    _encoder.classes_ = np.load(npy + '/' + column + '.npy')
    if type(value) == int:
        return str(_encoder.inverse_transform(value))  
    elif type(value) == list:
        return [str(_encoder.inverse_transform(v)) for v in value]
    else:
        return None

In [ ]:
inverse_transform([6,5],column='CSERVICE')
inverse_transform(6, 'CSTATUS')
inverse_transform(list(pd.unique(pfall['CSTATUS'])),'CSTATUS')

## status : was the message processed successfully ?

In [ ]:
def check_transformed(pfall,column,value):
    return pfall[column] == transform(value,column=column)

def assign_outcome_error(pfall):
    pfall = pfall.assign(error=(~( ( check_transformed(pfall,'CSTATUS','PENDING') & check_transformed(pfall,'CSERVICE','InvoicePortal')  ) | \
        (check_transformed(pfall,'CSTATUS','PENDING') & check_transformed(pfall,'CSERVICE','IDS')  ) | \
        check_transformed(pfall,'CSTATUS','SUCCESS') | \
        check_transformed(pfall,'CSTATUS','SUCCESS_DOWNLOADED') | check_transformed(pfall,'CSTATUS','SUCCESS_POLLQUEUE')  )).astype(int))
    return pfall

In [ ]:
pfall = assign_outcome_error(pfall)

In [ ]:
pfall[pfall['error'] == 1]

In [ ]:
#pfall = pfall.assign(outcome=(~( ((pfall['CSTATUS'] == 'PENDING') & (pfall['CSERVICE'] == 'InvoicePortal')) | ((pfall['CSTATUS'] == 'PENDING') & (pfall['CSERVICE'] == 'IDS')) | (pfall['CSTATUS'] == 'SUCCESS') | (pfall['CSTATUS'] == 'SUCCESS_DOWNLOADED') | (pfall['CSTATUS'] == 'SUCCESS_POLLQUEUE'))).astype(int))
    